In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import to_categorical
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tqdm import tqdm
%matplotlib inline
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
import os
from PIL import Image

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from numpy import asarray
from PIL import Image
x = []
y = []
filenames = (os.listdir("C:\\Users\\Admin\\Downloads\\mask\\with_mask"))
for i in filenames:
    photo = load_img("C:\\Users\\Admin\\Downloads\\mask\\with_mask\\"+i,target_size=(400,400))
    photo = img_to_array(photo)
    x.append(photo)
    y.append(1) # 1 with mask
filenames = (os.listdir("C:\\Users\\Admin\\Downloads\\mask\\without_mask"))
for i in filenames:
    photo = load_img("C:\\Users\\Admin\\Downloads\\mask\\without_mask\\"+i,target_size=(400,400))
    photo = img_to_array(photo)
    x.append(photo)
    y.append(0) # 0 without mask
x = asarray(x)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
print(x_train.shape, x_test.shape)

In [ ]:
#convert dependent variable to categorical
from keras.utils import to_categorical
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [ ]:
#normalize data
x_train = x_train/255.0
x_test =  x_test/255.0

In [ ]:
model = Sequential()
model.add(Conv2D(filters=32, kernel_size=(3, 3), activation="relu", input_shape=(400,400,3)))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.2))
model.add(Conv2D(filters=64, kernel_size=(3, 3), activation="relu"))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.22))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.2))
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
#model.add(Dropout(0.5))
#model.add(Dense(64, activation='relu'))
#model.add(Dropout(0.5))
model.add(Dense(2, activation='sigmoid'))

In [ ]:
model.compile(
  loss='binary_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [ ]:
#train model with training set
model_train = model.fit(x_train, y_train, batch_size=64,epochs=4,verbose=1,validation_split=0.20)

In [ ]:
import tensorflow as tf
from keras.models import load_model
model.save('maskdetection.h5')

In [ ]:
import tensorflow as tf

from keras.models import load_model
model = load_model('maskdetection.h5')
img = image.load_img('C:\\Users\\Admin\\Downloads\\pp2.jpg',target_size=(400,400,3))
img = image.img_to_array(img)

img = img/255

In [ ]:

proba = model.predict(img.reshape(1,400,400,3))


In [ ]:
proba

In [ ]:
res = ["Without Mask","With Mask"]
plt.figure()
plt.imshow(img)
plt.title("Predicted: {}".format(res[np.argmax(proba)]))

In [ ]:
import tensorflow as tf

from keras.models import load_model

model.save('maskdetection.h5')

In [ ]:
import tensorflow as tf
from keras.models import load_model
model = load_model('maskdetection.h5')

In [ ]:
import numpy as np
import cv2
face_cascade=cv2.CascadeClassifier('C:\\Users\\Admin\\Haarcascades\\haarcascade_frontalface_default.xml')

In [ ]:
def face_extractor(img):
    faces=face_cascade.detectMultiScale(img,1.3,5)
    if faces is():
        return None
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x-100,y-100),(x+w+100,y+h+100),(0,255,255),2)
        cropped_face=img[y:y+h,x:x+w]
    return cropped_face



In [ ]:
from keras.models import load_model
import cv2
import numpy as np

model = load_model('maskdetection.h5')
face_clsfr=cv2.CascadeClassifier('C:\\Users\\Admin\\Haarcascades\\haarcascade_frontalface_default.xml')

source=cv2.VideoCapture(0)

labels_dict={0:'without_mask',1:'with_mask'}
color_dict={0:(0,255,0),1:(0,0,255)}

while(True):

    ret,img=source.read()
    gray=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    faces=face_clsfr.detectMultiScale(img,1.3,5)  

    for x,y,w,h in faces:
    
        face_img=img[y-h:y+2*h,x-w:x+2*w]
        resized=cv2.resize(face_img,(400,400))
        #normalized=resized/255.0
        #reshaped=np.reshape(normalized,(1,400,400,1))
        #result=model.predict(reshaped)
        
        img = image.img_to_array(resized)
        img = img/255
        result=model.predict(img.reshape(1,400,400,3))

        label=np.argmax(result,axis=1)[0]
      
        cv2.rectangle(img,(x,y),(x+w,y+h),color_dict[label],2)
        cv2.rectangle(img,(x,y-40),(x+w,y),color_dict[label],-1)
        cv2.putText(
          img, labels_dict[label], 
          (x, y-10),
          cv2.FONT_HERSHEY_SIMPLEX,0.8,(255,255,255),2)
        
        
    cv2.imshow('LIVE',img)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()

In [ ]:
video_capture=cv2.VideoCapture(0)
while True:
    _,frame=video_capture.read()
    face=frame
    if type(face) is np.ndarray:
        face=cv2.resize(frame,(400,400))
        im=Image.fromarray(face,'RGB')
        #img_array=np.array(im)
        #img_array=np.expand_dims(img_array,axis=0)
        #pred=model.predict(img_array)
        #print(pred)
        img = image.img_to_array(im)
        img = img/255.0
        pred=model.predict(img.reshape(1,400,400,3))
        print(pred)
        res = ["Without Mask","With Mask"]
        #name="None matching"
        #name=format(res[np.argmin(pred)])
        if(pred[0][0]>pred[0][1]):
            name='without mask'
        else:
            name='with mask'
        cv2.putText(frame,name,(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    else:
        cv2.putText(frame,"No face found",(50,50),cv2.FONT_HERSHEY_COMPLEX,1,(0,255,0),2)
    cv2.imshow('Video',frame)
    key=cv2.waitKey(1)
    
    if(key==27):
        break
        
cv2.destroyAllWindows()
source.release()